# American Sign Language - Computer Vision Project

- Dataset: https://public.roboflow.com/object-detection/american-sign-language-letters
- Example Task: https://towardsdatascience.com/sign-language-recognition-with-advanced-computer-vision-7b74f20f3442

In [ ]:
# import numpy as np
# import tensorflow as tf

# # Set the seed for NumPy
# np.random.seed(42)

# # Set the seed for TensorFlow
# tf.random.set_seed(42)


In [ ]:
import numpy as np
import tensorflow as tf

# Set the seed for NumPy
np.random.seed(42)

# Set the seed for TensorFlow
tf.random.set_seed(42)

import os, glob
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf
from tensorflow.keras.utils import load_img, img_to_array, array_to_img
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.utils import to_categorical
tf.__version__


In [ ]:

# Custom functions: add to lesson notebook instead of file
%load_ext autoreload
%autoreload 2

import sys, os
# sys.path.append(os.path.abspath("../../"))

import ann_functions as af

In [ ]:
# Checking the contents of data folder
data_dir = "./American Sign Language Letters.v1-v1.multiclass/"
data_dir

In [ ]:
# Getting list of img file paths (ONLY, did not make recursuve so no folders)
img_files = glob.glob(data_dir+"**/*")#, recursive=True)
len(img_files)

In [ ]:
# Preview an example image (at full size)
img_loaded = load_img(img_files[0])
img_data = img_to_array(img_loaded)
img_data.shape

In [ ]:
## Set project-wide parameters
# # Saving image params as vars for reuse
BATCH_SIZE = 32
IMG_HEIGHT = 128
IMG_WIDTH = 128
TRAIN_SPLIT = 0.7  # Proportion of data for training
VAL_SPLIT = 0.15  # Proportion of data for validation (remaining will be for test)

In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array


In [ ]:

# Load the CSV file
csv_path = os.path.join(data_dir,"train","_classes.csv")
df = pd.read_csv(csv_path)

df = df.convert_dtypes()
# df['filename'] = df['filename']
df = df.set_index('filename')
df = df.astype(float)
df

In [ ]:
df.loc[:,'label'] = df.apply(lambda x: x.idxmax(), axis=1)
display(df.head(2))
df['label'].value_counts().sort_index()

In [ ]:
n_examples = df.drop(columns=['filename', 'label'], errors='ignore').sum()
n_examples

In [ ]:
label_cols = sorted(df.drop(columns=['filename','label'], errors='ignore').columns)
label_lookup = {i:label for i,label in enumerate(label_cols)}
label_lookup

In [ ]:
## Get the filepaths and labels
df = df.reset_index(drop=False)

df['filepath'] = df.loc[:,'filename'].astype(str).map(lambda x: os.path.join(data_dir, "train/", x)).values
filepaths = df['filepath']

labels = df[label_cols].astype(float).values
filepaths[0], labels[0]

In [ ]:
files_exist = np.array([os.path.exists(f) for f in filepaths])
files_exist.all()

In [ ]:
# labels

In [ ]:
display(load_img(filepaths[0]))
print(f"Letter: {label_lookup[np.argmax(labels[0])]}")

In [ ]:

# # Create a TensorFlow dataset from the image paths and labels
# def load_image(image_path, label, img_height=128, img_width=128):
#     target_size=(img_height, img_width)
#     image = load_img(image_path, target_size=target_size)
#     image = img_to_array(image)
#     image = image / 255.0  # Normalize the image
#     return image, label


# Function to load and preprocess images
def load_image(filename, label, img_height=128, img_width=128):
    img = tf.io.read_file(filename)
    # img = tf.image.decode_image(img, channels=3)
    img = tf.image.decode_image(img, channels=3, expand_animations=False)
    img.set_shape([None, None, 3])  # Explicitly set the shape
    img = tf.image.resize(img, [img_height, img_width])
    # img = img / 255.0  # Normalize the image
    return img, label


In [ ]:
ex_img, ex_label = load_image(filepaths[0], labels[0])
print(ex_img.shape)
display(array_to_img(ex_img))
print(f"Label: {ex_label}")
print(f"Label: {label_lookup[np.argmax(ex_label)]}")

#### Display Example of Each

In [ ]:
# ## Showing example of each letter
# label_lookup.values()

In [ ]:
# labels = sorted(df['label'].unique())
# labels

In [ ]:
eda_df = df[['filepath', 'label']]
eda_df

In [ ]:
### Plot example of each letter
ncols = 6
nrows = len(labels)//ncols + 1
fig, axes = plt.subplots(ncols=ncols, nrows=nrows, figsize=(10,6))
axes = axes.flatten()
unique_labels = sorted(eda_df['label'].unique())


for i, label in enumerate(unique_labels):
    fpath = eda_df.loc[ eda_df['label']==label,'filepath'].sample(1).values[0]
    
    loaded = plt.imread(fpath)
    axes[i].imshow(loaded)
    axes[i].set_title(label)
    axes[i].axis('off')
    

## remove unused axes
axes_labels_diff =  len(axes) - len(unique_labels)

if axes_labels_diff>0:
    for ax in axes[-axes_labels_diff:]:
        
        # difference = len(axes)
        fig.delaxes(ax=ax)   
        
fig.tight_layout()
    
    
    
    
    

In [ ]:
# df.loc[:,'label'] = df.loc[:,label_cols].apply(lambda x: x.idxmax(), axis=1)
# df['label'].value_counts().sort_index()

In [ ]:
# image_paths = np.array(image_paths)
# labels = np.array(labels)
# image_paths.shape, labels.shape

### Construct Train/Test/Val Tensorflow Datasets

In [ ]:
# load_image(image_paths[0], labels[0])
dataset = tf.data.Dataset.from_tensor_slices((filepaths, labels))

# Shuffle and batch the dataset
dataset = dataset.shuffle(buffer_size=len(dataset), reshuffle_each_iteration=False)


## Map the load_image function to the dataset
dataset = dataset.map(load_image,num_parallel_calls=tf.data.experimental.AUTOTUNE)

dataset


In [ ]:
# Determine split sizes
total_size = len(dataset)
train_size = int(TRAIN_SPLIT * total_size)
val_size = int(VAL_SPLIT * total_size)
test_size = total_size - train_size - val_size
print(f"{train_size=}, {test_size=}, {val_size=}")


In [ ]:
# Split the dataset
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size).take(val_size)
test_dataset = dataset.skip(train_size + val_size)

# # Cache the datset for faster access
# train_dataset = train_dataset.cache()
# val_dataset = val_dataset.cache()
# test_dataset = test_dataset.cache() 


In [ ]:
# Batch and prefetch the datasets
train_dataset = train_dataset.batch(BATCH_SIZE).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
val_dataset = val_dataset.batch(BATCH_SIZE).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

# Shuffle the trainin data
train_dataset = train_dataset.shuffle(buffer_size=train_dataset.cardinality(), 
                                      reshuffle_each_iteration=True) # DOUBLE CHECK BATCH_SIZE * 8


In [ ]:

# Use the datasets
for images, labels in train_dataset.take(1):
    print(f"Train batch - images: {images.shape}, labels: {labels.shape}")
    
for images, labels in val_dataset.take(1):
    print(f"Val batch - images: {images.shape}, labels: {labels.shape}")
    
    
for images, labels in test_dataset.take(1):
    print(f"Test batch - images: {images.shape}, labels: {labels.shape}")
    

In [ ]:
# Moedl from https://towardsdatascience.com/sign-language-recognition-with-advanced-computer-vision-7b74f20f3442
# from tensorflow
def make_model(name='towards-data-science',show_summary=False):
    model = models.Sequential(name=name)
    model.add(layers.Rescaling(1./255 , input_shape = (IMG_HEIGHT,IMG_WIDTH,3)))
    model.add(layers.Conv2D(75 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu' ))#, input_shape = (28,28,1)))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPool2D((2,2) , strides = 2 , padding = 'same'))
    model.add(layers.Conv2D(50 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
    model.add(layers.Dropout(0.2))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPool2D((2,2) , strides = 2 , padding = 'same'))
    model.add(layers.Conv2D(25 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPool2D((2,2) , strides = 2 , padding = 'same'))
    model.add(layers.Flatten())
    model.add(layers.Dense(units = 512 , activation = 'relu'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(units = len(label_lookup   ) , activation = 'softmax'))
    
    model.compile(optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics = ['accuracy'])
    if show_summary:
        model.summary()
    return model


def get_callbacks(monitor='val_accuracy', patience=15, start_from_epoch=3, restore_best_weights=False):
    early_stopping = tf.keras.callbacks.EarlyStopping(patience=patience,start_from_epoch=start_from_epoch,
                                                      monitor=monitor,
                                                      restore_best_weights=restore_best_weights)
    return [early_stopping]


model = make_model(show_summary=True)


In [ ]:
model = make_model()
history = model.fit(train_dataset,epochs = 100 ,validation_data = val_dataset, callbacks=get_callbacks())
af.evaluate_classification_network(model,X_test=test_dataset,history=history, figsize=(15,15),target_names=label_lookup.values());

In [ ]:

def make_model2(name='CNN1',show_summary=False):
    
    model = models.Sequential(name=name)
    # Using rescaling layer to scale pixel values
    model.add(layers.Rescaling(1./255 , input_shape = (IMG_HEIGHT,IMG_WIDTH,3)))
    
    # Convolutional layer
    model.add(
        layers.Conv2D(
            filters=16,  # How many filters you want to use
            kernel_size=3, # size of each filter
            # input_shape=input_shape,
            padding='same')) 
    # Pooling layer
    model.add(layers.MaxPooling2D(pool_size=2))  # Size of pooling


    # Convolutional layer
    model.add(
        layers.Conv2D(
            filters=32,#64,  # How many filters you want to use
            kernel_size=3,  # size of each filter
            # input_shape=input_shape,
            padding='same')) 
    # Pooling layer
    model.add(layers.MaxPooling2D(pool_size=2))  # Size of pooling
    
    # Flattening layer
    model.add(layers.Flatten())
    # Output layer
    model.add(
        layers.Dense(len(label_lookup), activation="softmax") )  
    ## Adding learning rate decay
    lr_schedule = optimizers.schedules.ExponentialDecay(
        initial_learning_rate=0.01, decay_steps=10000, decay_rate=0.95
    )  # 0.9)
    optimizer = optimizers.legacy.Adam(learning_rate=lr_schedule)
    
    model.compile(optimizer=optimizer, 
                  loss=tf.keras.losses.CategoricalCrossentropy(),
                  metrics=['accuracy'])
    if show_summary:
        model.summary()
    return model

In [ ]:
model2 = make_model2()
history2 = model2.fit(train_dataset,epochs = 100 ,validation_data = val_dataset, callbacks=get_callbacks())
af.evaluate_classification_network(model2,X_test=test_dataset,history=history2, figsize=(15,15));